<a href="https://colab.research.google.com/github/y-arjun-y/cloudclassifier/blob/main/cloudclassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## [Cloud Classifier Project for fast.ai - Practical Deep Learning for Coders - Lesson 2](https://github.com/y-arjun-y/cloudclassifier)

### Imports (it's normal for it to request access to GDrive for Desktop)

In [ ]:
# !pip install fastbook==0.0.17
# !pip uninstall tornado -y
# !yes | pip install tornado==5.1.0

import fastbook
fastbook.setup_book()

In [5]:
from fastbook import *
from fastai.vision.widgets import *

### Download dataset (if you don't have a Bing Image Search API Key)

In [ ]:
!npx degit y-arjun-y/cloudclassifier -f

In [6]:
cloud_types = 'cumulus cloud', 'stratus cloud', 'cirrus cloud'
path = Path('clouds')

### Bing Image Search API (optional if you downloaded the dataset from the repository)

In [ ]:
key = os.environ.get('AZURE_SEARCH_KEY', 'BING_IMAGE_SEARCH_API_KEY')

In [ ]:
# Using only three basic cloud types. Feel free to download more!
cloud_types = 'cumulus cloud', 'stratus cloud', 'cirrus cloud'
path = Path('clouds')

In [ ]:
if not path.exists():
    path.mkdir()
    for o in cloud_types:
        dest = (path/o)
        dest.mkdir(exist_ok=True)
        results = search_images_bing(key, f'{o}')
        download_images(dest, urls=results.attrgot('contentUrl'))

In [ ]:
failed = verify_images(get_image_files(path))
failed.map(Path.unlink)

### DataLoaders and ResNet-18

In [7]:
clouds = DataBlock(
    blocks=(ImageBlock, CategoryBlock),
    get_items=get_image_files,
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=Resize(128)
)

In [8]:
clouds = clouds.new(
    item_tfms=RandomResizedCrop(224, min_scale=0.5),
    batch_tfms=aug_transforms()
)
dls = clouds.dataloaders(path)

In [ ]:
learn = cnn_learner(dls, resnet18, metrics=error_rate)
learn.fine_tune(1)

### Exporting the model (you need to run this)

In [10]:
learn.export()

In [ ]:
path = Path()
path.ls(file_exts='.pkl')

### Supplying your own cloud image and getting a prediction!

In [12]:
learn_inf = load_learner(path/'export.pkl')

In [ ]:
btn_upload = widgets.FileUpload()
btn_upload

In [14]:
img = PILImage.create(btn_upload.data[-1])

In [15]:
out_pl = widgets.Output()

In [ ]:
pred, pred_idx, probs = learn_inf.predict(img)

In [ ]:
lbl_pred = widgets.Label()
lbl_pred.value = f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}'
lbl_pred